# Computing number densities of CMASS from published samples

In [18]:
import fitsio
import numpy as np
import matplotlib.pyplot as plt

In [133]:
# published CMASS catalogues 
cmass_sgc = fitsio.read('cmass_cat/galaxy_DR12v5_CMASS_South.fits.gz')
cmass_ngc = fitsio.read('cmass_cat/galaxy_DR12v5_CMASS_North.fits.gz')
# this is unpublished version - contain all photometric targets 
# http://arxiv.org/abs/1509.06529 
cmass_photo = fitsio.read('cmass_cat/cmass-dr12v4-S-Reid-full.dat.fits')

In [134]:
# No redshift cut
zmin=0.0
zmax=10
cmass_zcut = cmass_sgc[(cmass_sgc['Z'] > zmin) & (cmass_sgc['Z'] < zmax)]

Production of this paper http://arxiv.org/abs/1509.06529  \
But the numbers don't exactly match with table 2. \
Number in the paper is 258,901.

In [138]:
print ('Number of photometric sources: N_photo=', cmass_photo.size)

Number of photometric sources: N_photo= 258884


Now let's compute number of various sources from the spectroscopic samples.\
I am going to compute: 
* $N_{\rm source}$: the number of sources in the sample\
* $N_{\rm cp}$: the number of missing galaxies due to fiber collision. Those galaxies are in the photometric sample but not in the spectroscopic sample. Nearby galaxies with spectroscopic redshift are double-counted to account for those missing galaxies. 
* $N_{\rm nz}$: the number of galaxies with redshift failure. 
* $N_{\rm photo}$: the number of sources I beilieve in photometric sample for training. Clean galaxies + double counted fiber-collosion galaxies + galaxies upweighted to account for redshift failure of nearby galaxies.
* $N_{\rm eff}$: effective number of galaxies that systematics are corrected (seeing and stellar density) + double counting of fiber-collision galaxies + redshift failure correction



There are three `area` described in Reid et al. 2016.
* Total area: total area of BOSS survey. The area is constructed of spherical polygons
* Veto area: Area vetoed by following reasons as below. Those areas are already vetoed in randoms. 
1. centerpost : each sloan plate is secured to the focal plane by a central bolt. Targets coninciding with the centerpost cannot be observed (0.04% region) 
2. fiber collision : Ly-alpha quasar targets receive higher priority than galaxies. In regions of only a single spectroscopic tile, galaxies within a fiber collision radius of those targets are not observable. Masking a 62'' radius around each quasar. 1.5% survey area
3. Bright stars mask + Bright objects mask 
4. Non-photometric condition amsk: mask regions where the imaging was not photometric in g,r,or i bands, the PSF modeling failed, the imaging reduction pipeline fails to process. 3.4%
5. Seeing cut 
6. Extinction cut. 

* Used area: Total area - Veto area


In [153]:
def comp_various_numbers( cmass_zcut ):
    
    #redshift cut
    #cmass_zcut = cmass_sgc[(cmass_sgc['Z'] > zmin) & (cmass_sgc['Z'] < zmax)]
    
    # Number of sources in the sample
    print ('Number of spectroscopic sample\n')
    print ('Number of source: N_source=', cmass_zcut.size)

    # Number of fiber collision galaxies (should be double counted)
    print ('Number of fiber-collision: N_cp=', np.sum(cmass_zcut['WEIGHT_CP'] == 2))

    # Number of galaxies with redshift failure 
    print ('Number of redshift failure: N_nz=', np.sum(cmass_zcut['WEIGHT_NOZ'] != 1))
    #print ('N_source + fiber-collision =', cmass_zcut.size + np.sum(cmass_zcut['WEIGHT_CP'] == 2))

    # Number of sources that I believe in photoemtric sample for training 
    # photometric sample has everything 
    print ('Number of photometric sources : N_photo =', np.sum( (cmass_zcut['WEIGHT_CP'] + cmass_zcut['WEIGHT_NOZ'] - 1)))

    # the number of galaxies weighted by systematics weight + fiber collision double counting + redshift failure correction
    Neff = np.sum( cmass_zcut['WEIGHT_SYSTOT']  * (cmass_zcut['WEIGHT_CP'] + cmass_zcut['WEIGHT_NOZ'] - 1))
    print ('N_effective =', Neff)
    return Neff
    

Areas obtained from the same paper, table 2. 

## SGC

In [154]:
Neff = comp_various_numbers( cmass_sgc )
# number density with Area from literature
Aeff = 2525
Aused = 2560
Atotal = 2823
print ('\nnumber density with effective area :' , Neff/Aeff)
print ('number density with used area :' , Neff/Aused)
print ('number density with total area :' , Neff/Atotal)

Number of spectroscopic sample

Number of source: N_source= 230831
Number of fiber-collision: N_cp= 8475
Number of redshift failure: N_nz= 4760
Number of photometric sources : N_photo = 246794.53
N_effective = 249701.58

number density with effective area : 98.89171410891089
number density with used area : 97.53967895507813
number density with total area : 88.4525604410202


## NGC

In [131]:
Neff = comp_various_numbers( cmass_ngc )
# number density with Area from literature
Aeff = 6851
Aused = 6934
Atotal = 7429
print ('\nnumber density with effective area :' , Neff/Aeff)
print ('number density with used area :' , Neff/Aused)
print ('number density with total area :' , Neff/Atotal)

Number of spectroscopic sample

Number of source: N_source= 618806
Number of fiber-collision: N_cp= 26856
Number of redshift failure: N_nz= 9608
Number of photometric sources : N_photo = 662470.4
N_effective = 658792.5

number density with effective area : 96.16004962779157
number density with used area : 95.00901355638881
number density with total area : 88.67848970251717
